In [1]:
import pandas as pd
import numpy as np
import re

# Generación del dataset final
## Importamos primero los datasets originales

In [2]:
ds_asf = pd.read_csv(
    'https://raw.githubusercontent.com/Erick-INCS/Corruption-analysis/main/datasets/asf_2017-2019_2.csv',
     encoding='latin-1') #dataset de corrupcion

ds_cpi = pd.read_csv('https://raw.githubusercontent.com/Erick-INCS/Bedu-py/main/CPI_Mex_full.csv') #dataset de indice de corrupcion

ds_pres = pd.read_parquet('https://github.com/Erick-INCS/Corruption-analysis/blob/main/datasets/presupuesto_2017-2020.parquet?raw=true') #dataset de presupuesto


In [3]:
ds_asf.head(2)

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Número,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Clave Acción,Acción Superveniente,Texto Acción,Estado de Trámite,Auditorías,Acciones
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,2019-1-19GYR-19-1393-06-001,-,"Se presume un probable daño o perjuicio, o amb...",En seguimiento,1,1
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-001,-,La Auditoría Superior de la Federación emite l...,Con seguimiento concluido,1,1


In [4]:
ds_cpi.head()

,estado,municipio,Accesibilidad al espacio público abierto,Acceso a Internet,Acceso a agua mejorada,Aglomeración económica,CALIDAD DE VIDA,Calidad del aire,Capacidad institucional y ﬁnanzas municipales,Coeﬁciente de Gini,Concentraciones de material particulado,Concentración de CO2,Crecimiento económico,Densidad de la interconexión vial,Densidad de médicos,Densidad económica,Densidad poblacional,Densidad vial,Desempleo juvenil,Deuda subnacional,EQUIDAD E INCLUSIÓN SOCIAL,Educación,Empleo,Energía,Equidad económica,Espacio habitable suﬁciente,Espacio público,Esperanza de vida al nacer,Eﬁciencia del gasto local,Eﬁciencia en el uso de suelo,Fatalidades de tránsito,Forma urbana,GOBERNANZA Y LEGISLACIÓN URBANA,Gobernanza de la urbanización,INFRAESTRUCTURA DE DESARROLLO,Inclusión de género,Inclusión social,Infraestructura de comunicaciones,Infraestructura de vivienda,Infraestructura social,Inscripción equitativa en educación a nivel secundario,Longitud de transporte masivo,Manejo de residuos,Movilidad urbana,Número de estaciones de monitoreo,PRODUCTIVIDAD,Participación electoral,Participación y rendición de cuentas,Producto urbano per cápita,Promedio de años de escolaridad,Proporción de generación de energía renovable,Recaudación de ingresos propios,Recolección de residuos sólidos,Relación de dependencia de la tercera edad,Relación empleo-población,SOSTENIBILIDAD AMBIENTAL,Salud,Seguridad y protección,Superﬁcie destinada a vías,Tasa de alfabetización,Tasa de desempleo,Tasa de homicidios,Tasa de mortalidad de menores de 5 años,Tasa de pobreza,Tratamiento de aguas residuales,Velocidad de banda ancha promedio,Vivienda durable,Viviendas en barrios precarios,Áreas verdes per cápita
0,Aguascalientes,Aguascalientes,73.28,27.92,99.02,68.10,69.72,62.83,43.60,49.45,67.43,46.08,51.88,100.0,71.24,68.10,42.90,84.57,62.88,14.66,72.49,85.15,64.44,0.0,47.80,100.0,63.96,73.16,91.34,0.00,84.71,89.49,29.10,0.00,64.85,90.33,79.34,35.68,85.48,71.24,90.33,0.0,41.47,42.36,75.0,61.48,43.70,43.70,41.23,73.79,0.0,24.80,2.94,62.54,58.77,34.77,67.44,62.34,83.91,96.51,70.12,62.34,61.71,46.15,80.00,43.44,99.99,95.80,54.65
1,Aguascalientes,Jesús María,38.22,32.03,97.96,70.14,60.84,56.75,20.87,53.62,67.42,46.08,59.68,100.0,21.70,70.14,10.60,78.63,74.44,3.62,73.72,71.44,69.28,0.0,51.67,100.0,38.48,74.97,43.66,100.00,81.19,80.49,56.84,100.00,52.60,85.28,84.21,43.06,77.14,21.70,85.28,0.0,83.20,40.59,NaN,66.37,49.65,49.65,48.97,47.14,0.0,15.33,99.36,70.39,57.34,69.97,73.82,59.62,62.82,95.74,81.23,59.62,72.67,49.71,67.03,54.10,100.00,93.98,38.73
2,Aguascalientes,San Francisco de los Romo,100.00,8.40,98.42,65.86,62.71,56.75,37.20,60.70,67.43,46.08,60.89,100.0,27.90,65.86,12.31,87.82,69.52,0.00,72.67,75.79,64.59,0.0,47.86,100.0,58.96,73.16,100.00,0.00,77.64,88.00,29.81,0.00,51.66,87.23,82.93,25.92,77.68,27.90,87.23,0.0,35.36,38.82,NaN,63.78,52.23,52.23,51.47,57.57,0.0,11.61,2.24,70.31,55.52,30.70,64.17,51.93,76.19,94.02,73.66,51.93,55.17,35.02,68.48,43.44,100.00,96.33,17.93
3,Baja California,Ensenada,28.97,45.03,85.26,57.58,54.24,64.29,44.57,45.63,82.50,46.08,42.72,100.0,50.46,57.58,16.13,74.58,78.69,57.96,72.79,72.25,63.01,0.0,44.34,100.0,27.56,69.20,61.59,46.85,93.74,81.22,42.42,46.85,56.56,92.33,81.71,49.57,54.68,50.46,92.33,0.0,89.10,46.87,NaN,54.44,35.83,35.83,27.83,49.14,0.0,14.15,91.14,57.61,51.18,76.70,67.96,49.20,69.07,95.35,74.85,49.20,66.72,43.04,87.07,54.10,17.34,84.74,26.14
4,Baja California,Mexicali,42.90,37.96,95.59,55.86,59.45,35.36,66.59,48.62,0.00,46.08,51.07,95.5,59.54,55.86,12.96,75.59,69.51,59.07,74.23,85.32,70.31,100.0,48.49,100.0,29.74,66.38,100.00,0.00,99.33,79.94,29.64,0.00,57.87,92.00,82.18,40.70,59.52,59.54,92.00,0.0,97.71,49.66,60.0,59.08,22.34,22.34,40.55,74.43,100.0,40.70,95.51,61.60,62.73,77.69,62.00,60.72,68.72,96.22,77.89,60.72,57.62,48.37,99.91,43.44,29.52,94.85,16.58


In [5]:
ds_cpi2 = ds_cpi.groupby('estado').mean()
ds_cpi2 = ds_cpi2.reset_index()
ds_cpi2= ds_cpi2.set_index('estado').mean(axis=1).reset_index()

In [6]:
ds_asf['a'] =ds_asf['Ente Fiscalizado'] 
ds_asf['a']=ds_asf['a'].str[13:].str.split(',').apply(lambda l : l[0]).str.strip() #obtenemos los municipios disponibles
ds_final = pd.merge(ds_asf,ds_cpi,left_on='a',right_on='municipio',how='left')
# ds_final.head()
ds_final['mean_cpi'] = ds_final.iloc[:,20:-1].mean(axis=1)
ds_final = ds_final.drop(columns=ds_final.columns[19:-1])
ds_final.head(2)

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Número,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Clave Acción,Acción Superveniente,Texto Acción,Estado de Trámite,Auditorías,Acciones,a,mean_cpi
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,2019-1-19GYR-19-1393-06-001,-,"Se presume un probable daño o perjuicio, o amb...",En seguimiento,1,1,icano del Seguro Social,NaN
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-001,-,La Auditoría Superior de la Federación emite l...,Con seguimiento concluido,1,1,icano del Seguro Social,NaN


In [7]:
def renom_estados(name):
  estados = {
    'Michoacán de Ocampo': 'Michoacán', 
    'México': 'Estado de México', 
    'Veracruz Ignacio de la Llave': 'Veracruz',
    'Veracruz de Ignacio de la Llave': 'Veracruz'
    # 'Coahuila de Zaragoza':'Coahuila',
    }

  if name in estados:
    return estados[name]
  
  return name

In [8]:
ds_final_copia = ds_final.copy()
ds_final_copia['Entidad Federativa']=ds_final_copia['Entidad Federativa'].map(renom_estados);
ds_pres['ENTIDAD_FEDERATIVA'] = ds_pres['ENTIDAD_FEDERATIVA'].map(renom_estados);

In [9]:
ds_final = pd.merge(ds_final_copia,ds_cpi2,left_on='Entidad Federativa', right_on='estado',how='left')
ds_final['mean_cpi']= ds_final['mean_cpi'].fillna(ds_final.iloc[:,-1])
ds_final = ds_final.drop(columns=ds_final.columns[-2:])
ds_final.head()

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Número,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Clave Acción,Acción Superveniente,Texto Acción,Estado de Trámite,Auditorías,Acciones,a,mean_cpi
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,2019-1-19GYR-19-1393-06-001,-,"Se presume un probable daño o perjuicio, o amb...",En seguimiento,1,1,icano del Seguro Social,61.295084
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-001,-,La Auditoría Superior de la Federación emite l...,Con seguimiento concluido,1,1,icano del Seguro Social,61.295084
2,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-004,-,La Auditoría Superior de la Federación emite l...,En seguimiento,1,1,icano del Seguro Social,61.295084
3,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-002,-,La Auditoría Superior de la Federación emite l...,Con seguimiento concluido,1,1,icano del Seguro Social,61.295084
4,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,1393,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,2019-9-19GYR-19-1393-08-003,-,La Auditoría Superior de la Federación emite l...,Con seguimiento concluido,1,1,icano del Seguro Social,61.295084


In [10]:
ds_final = ds_final.drop(['Número', 'Clave Acción', 'Acción Superveniente', 
              'Estado de Trámite', 
              'Auditorías', 'Acciones'],
        axis=1)

In [11]:
def get_money(text):

  if pd.isna(text):    
    return np.nan
  re_dinero = re.compile(r'monto de ([\d|,|\.]+) pesos')
  res = re_dinero.search(text)
  if res:
    re_num = re.compile(r'[^0-9]')
    return float(re.sub(re_num, '', res.groups()[0][:-3]))
  return np.nan

In [12]:
ds_final['desaparecido'] = ds_final['Texto Acción'].apply(get_money)
ds_final.head(2)

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Texto Acción,a,mean_cpi,desaparecido
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,"Se presume un probable daño o perjuicio, o amb...",icano del Seguro Social,61.295084,1.342933e+09
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN


In [13]:
ds_pres.head(2)

,CICLO_RECURSO,PROGRAMA_FONDO_CONVENIO_ESPECIFICO,DESC_PARTIDA_GENERICA,DESC_TIPO_GASTO,ID_RAMO,MONTO_RECAUDADO,TRIMESTRE,MONTO_APROBADO,ID_PARTIDA_GENERICA,MONTO_EJERCIDO,MONTO_PAGADO,MONTO_DEVENGADO,MONTO_COMPROMETIDO,MONTO_MODIFICADO,TIPO_RECURSO,CICLO,DESC_RAMO,ENTIDAD_FEDERATIVA,MUNICIPIO,INSTITUCION_EJECUTORA
0,2008,FAM 2008,Transferencias otorgadas a entidades paraestat...,Gasto de Inversión,33,486921.0,1,486921.0,421,457224.0,457224.0,457224.0,460560.0,486921.0,"FEDERALES (APORTACIONES, SUBSIDIOS Y CONVENIOS)",2017,Aportaciones Federales para Entidades Federati...,Aguascalientes,Gobierno de la Entidad,FISICA EDUCATIVA INSTITUTO DE INFRAESTRUCTURA
1,2010,FAM 2010,Transferencias otorgadas a entidades paraestat...,Gasto de Inversión,33,1526132.0,1,1526132.0,421,852000.0,852000.0,852000.0,1457215.0,1526132.0,"FEDERALES (APORTACIONES, SUBSIDIOS Y CONVENIOS)",2017,Aportaciones Federales para Entidades Federati...,Aguascalientes,Gobierno de la Entidad,FISICA EDUCATIVA INSTITUTO DE INFRAESTRUCTURA


In [14]:
ds_pres_mun = ds_pres.copy().groupby(['MUNICIPIO','CICLO_RECURSO'])['MONTO_MODIFICADO','MONTO_COMPROMETIDO'].sum()
ds_pres_est = ds_pres.copy().groupby(['ENTIDAD_FEDERATIVA','CICLO_RECURSO'])['MONTO_MODIFICADO','MONTO_COMPROMETIDO'].sum()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [15]:
ds_pres_mun=ds_pres_mun.reset_index()
ds_pres_est=ds_pres_est.reset_index()

In [16]:
peña = 2014
ds_pres_mun = ds_pres_mun[(ds_pres_mun['CICLO_RECURSO'] >= peña) & (ds_pres_mun['CICLO_RECURSO'] <= 2020)]
ds_pres_mun['es_peña'] = ds_pres_mun['CICLO_RECURSO'] < 2019;

In [17]:
ds_pres_est = ds_pres_est[(ds_pres_est['CICLO_RECURSO'] >= peña) & (ds_pres_est['CICLO_RECURSO'] <= 2020)]
ds_pres_est['es_peña'] = ds_pres_est['CICLO_RECURSO'] < 2019;

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
ds_pres_mun.head()

,MUNICIPIO,CICLO_RECURSO,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña
5,Abalá,2014,NaN,NaN,True
6,Abalá,2015,NaN,NaN,True
7,Abalá,2016,NaN,NaN,True
8,Abalá,2017,NaN,NaN,True
9,Abalá,2018,6106576.19,6106576.19,True


In [19]:
len(ds_pres_est[ds_pres_est['CICLO_RECURSO']==2016]['ENTIDAD_FEDERATIVA'].unique().tolist())

32

In [20]:
algo = pd.merge(ds_final,ds_pres_mun,how='left',left_on=['Año Cuenta Pública','a'],right_on=['CICLO_RECURSO','MUNICIPIO'])
algo2 = algo.copy()

In [21]:
algo2.head(3)

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Texto Acción,a,mean_cpi,desaparecido,MUNICIPIO,CICLO_RECURSO,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,"Se presume un probable daño o perjuicio, o amb...",icano del Seguro Social,61.295084,1.342933e+09,NaN,NaN,NaN,NaN,NaN
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
ds_pres_est[ds_pres_est['ENTIDAD_FEDERATIVA'] == 'Guerrero']

,ENTIDAD_FEDERATIVA,CICLO_RECURSO,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña
137,Guerrero,2014,NaN,NaN,True
138,Guerrero,2015,NaN,NaN,True
139,Guerrero,2016,NaN,NaN,True
140,Guerrero,2017,9.865508e+09,9.803284e+09,True
141,Guerrero,2018,4.139614e+10,3.882868e+10,True
142,Guerrero,2019,2.063604e+11,1.563516e+11,False
143,Guerrero,2020,9.152403e+10,4.763362e+10,False


In [23]:
algo3 = pd.merge(algo2,ds_pres_est,how='left',left_on=['Año Cuenta Pública','Entidad Federativa'],right_on=['CICLO_RECURSO','ENTIDAD_FEDERATIVA'],suffixes=('','_y'))

In [24]:
e =set(ds_pres_est['ENTIDAD_FEDERATIVA'].unique().tolist())

In [25]:
d =set(algo2['Entidad Federativa'].unique().tolist())

e - d

set()

In [26]:
# ds_pres_est['ENTIDAD_FEDERATIVA'].unique().tolist()

In [27]:
algo3

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Texto Acción,a,mean_cpi,desaparecido,MUNICIPIO,CICLO_RECURSO,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña,ENTIDAD_FEDERATIVA,CICLO_RECURSO_y,MONTO_MODIFICADO_y,MONTO_COMPROMETIDO_y,es_peña_y
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,"Se presume un probable daño o perjuicio, o amb...",icano del Seguro Social,61.295084,1.342933e+09,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2019,1.317958e+11,8.079752e+10,False
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2019,1.317958e+11,8.079752e+10,False
2,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2019,1.317958e+11,8.079752e+10,False
3,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2019,1.317958e+11,8.079752e+10,False
4,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,icano del Seguro Social,61.295084,NaN,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2019,1.317958e+11,8.079752e+10,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34107,2020,Gasto Federalizado,"Ramos Generales 23, 33 y Otros",Sistema para el Desarrollo Integral de la Fami...,Michoacán,De Cumplimiento,Cumplimiento de las Disposiciones Establecidas...,-,-,Sistema para el Desarrollo Integral de la Fami...,Solicitud de Aclaración,Para que el Sistema para el Desarrollo Integra...,el Desarrollo Integral de la Familia Michoacana,54.712334,NaN,NaN,NaN,NaN,NaN,NaN,Michoacán,2020,1.338753e+11,6.262678e+10,False
34108,2020,Gasto Federalizado,Seguridad y Protección Ciudadana,Secretariado Ejecutivo del Sistema Nacional de...,Ciudad de México,De Cumplimiento,Recursos del Fondo de Aportaciones para la Seg...,-,-,Secretariado Ejecutivo del Sistema Nacional de...,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,Ejecutivo del Sistema Nacional de Seguridad Pú...,61.295084,NaN,NaN,NaN,NaN,NaN,NaN,Ciudad de México,2020,7.967678e+10,2.905451e+10,False
34109,2020,Gasto Federalizado,Seguridad y Protección Ciudadana,Secretariado Ejecutivo del Sistema Nacional de...,Ciudad de México,De Cumplimiento,Recursos del Fondo de Aportaciones para la Seg...,-,-,Secretariado Ejecutivo del Sistema Nacional de...,Promoción de Responsabilidad Administrativa Sa...,La Auditorí

In [28]:
for i in range(16,20):
  algo3.iloc[:,i] = algo3.iloc[:,i].fillna(algo3.iloc[:,i+5])
# algo3.iloc[:,16]
# algo3.iloc[:,16+5]

In [29]:
algo3 = algo3.drop(columns=algo3.columns[20:])
algo3 = algo3.drop(columns=['MUNICIPIO','CICLO_RECURSO','a'])


In [31]:
algo3

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Texto Acción,mean_cpi,desaparecido,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña
0,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Pliego de Observaciones,"Se presume un probable daño o perjuicio, o amb...",61.295084,1.342933e+09,1.317958e+11,8.079752e+10,False
1,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,1.317958e+11,8.079752e+10,False
2,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,1.317958e+11,8.079752e+10,False
3,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,1.317958e+11,8.079752e+10,False
4,2019,Gasto Federalizado,Aportaciones A Seguridad Social,Instituto Mexicano del Seguro Social,Ciudad de México,De Cumplimiento,Programa de Atención a la Salud y Medicamentos...,"1,342,933.00","1,342,933.00",Instituto Mexicano del Seguro Social,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,1.317958e+11,8.079752e+10,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34107,2020,Gasto Federalizado,"Ramos Generales 23, 33 y Otros",Sistema para el Desarrollo Integral de la Fami...,Michoacán,De Cumplimiento,Cumplimiento de las Disposiciones Establecidas...,-,-,Sistema para el Desarrollo Integral de la Fami...,Solicitud de Aclaración,Para que el Sistema para el Desarrollo Integra...,54.712334,NaN,1.338753e+11,6.262678e+10,False
34108,2020,Gasto Federalizado,Seguridad y Protección Ciudadana,Secretariado Ejecutivo del Sistema Nacional de...,Ciudad de México,De Cumplimiento,Recursos del Fondo de Aportaciones para la Seg...,-,-,Secretariado Ejecutivo del Sistema Nacional de...,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,7.967678e+10,2.905451e+10,False
34109,2020,Gasto Federalizado,Seguridad y Protección Ciudadana,Secretariado Ejecutivo del Sistema Nacional de...,Ciudad de México,De Cumplimiento,Recursos del Fondo de Aportaciones para la Seg...,-,-,Secretariado Ejecutivo del Sistema Nacional de...,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,7.967678e+10,2.905451e+10,False
34110,2020,Gasto Federalizado,Seguridad y Protección Ciudadana,Secretariado Ejecutivo del Sistema Nacional de...,Ciudad de México,De Cumplimiento,Recursos del Fondo de Aportaciones para la Seg...,-,-,Secretariado Ejecutivo del Sistema Nacional de...,Promoción de Responsabilidad Administrativa Sa...,La Auditoría Superior de la Federación emite l...,61.295084,NaN,7.967678e+10,2.905451e+10,False


In [30]:
algo3[algo3['MONTO_MODIFICADO'].isna()]

,Año Cuenta Pública,Grupo Funcional,Sector,Ente Fiscalizado,Entidad Federativa,Tipo Auditoría,Título,Universo (miles pesos),Muestra (miles pesos),Ente a quien se dirige la Acción,Tipo Acción,Texto Acción,mean_cpi,desaparecido,MONTO_MODIFICADO,MONTO_COMPROMETIDO,es_peña
13631,2016,Gasto Federalizado,Instituciones Públicas de Educación Superior,Universidad Autónoma de Guerrero,Guerrero,Financiera y de Cumplimiento,Subsidios Federales para Organismos Descentral...,"1,780,731.40","1,765,637.30",Gobierno del Estado de Guerrero,Pliego de Observaciones,Se presume un probable daño o perjuicio o ambo...,52.394829,561493.0,NaN,NaN,True
13632,2016,Gasto Federalizado,Instituciones Públicas de Educación Superior,Universidad Autónoma de Guerrero,Guerrero,Financiera y de Cumplimiento,Subsidios Federales para Organismos Descentral...,"1,780,731.40","1,765,637.30",Universidad Autónoma de Guerrero,Recomendación,Para que la Universidad Autónoma de Guerrero p...,52.394829,NaN,NaN,NaN,True
15040,2016,Gasto Federalizado,"Ramos Generales 23, 33 y Otros",Gobierno del Estado de Guerrero,Guerrero,Financiera y de Cumplimiento,Apoyos a Centros y Organizaciones de Educación,"590,156.40","512,626.80",Gobierno del Estado de Guerrero,Informe de Presunta Responsabilidad Administra...,En virtud de la no solventación del Pliego de ...,52.394829,NaN,NaN,NaN,True
15041,2016,Gasto Federalizado,"Ramos Generales 23, 33 y Otros",Gobierno del Estado de Guerrero,Guerrero,Financiera y de Cumplimiento,Apoyos a Centros y Organizaciones de Educación,"590,156.40","512,626.80",Gobierno del Estado de Guerrero,Pliego de Observaciones,Se presume un probable daño o perjuicio o ambo...,52.394829,68775.0,NaN,NaN,True
15042,2016,Gasto Federalizado,"Ramos Generales 23, 33 y Otros",Gobierno del Estado de Guerrero,Guerrero,Financiera y de Cumplimiento,Apoyos a Centros y Organizaciones de Educación,"590,156.40","512,626.80",Gobierno del Estado de Guerrero,Pliego de Observaciones,NaN,52.394829,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32798,2014,Gasto Federalizado,Salud,Comisión Nacional de Protección Social en Salud,Ciudad de México,Financiera y de Cumplimiento,PROSPERA Programa de Inclusión Social (Compone...,-,-,Instituto de Salud para el Bienestar,Recomendación,Para que la Comisión Nacional de Protección So...,61.295084,NaN,NaN,NaN,True
32799,2014,Gasto Federalizado,Salud,Secretaría de Salud,Ciudad de México,Financiera y de Cumplimiento,Recursos del Fondo de Aportaciones para los Se...,-,-,-,-,-,61.295084,NaN,NaN,NaN,True
32800,2014,Gasto Federalizado,Salud,Secretaría de Salud,Ciudad de México,Financiera y de Cumplimiento,Programa de Apoyo para Fortalecer la Calidad e...,-,-,Secretaría de Salud,Recomendación,"Para que la Secretaría de Salud, instruya a qu...",61.295084,NaN,NaN,NaN,True
32801,2014,Gasto Federalizado,Salud,Secretaría de Salud,Ciudad de México,Financiera y de Cumplimiento,Participación Social en el Sector Salud (Avale...,-,-,-,-,-,61.295084,NaN,NaN,NaN,True
